In [37]:
import random

def isprime(n):
    if n%2 == 0:
        return False
    else:
        i = 3
        while i**2 < n:
            if n% i == 0:
                return False
            i +=2
        return True

def sampleprime(n):
    while True:
        p = random.randint(1, 2**n)
        if isprime(p):
            return p

The second issue is that RSA (as presented above) is not secure. Cryptography has a minimal acceptable notion of security known as Indistinguishability under Chosen-Plaintext Attack. Among other things, this implies that you cannot tell if two ciphertexts are encryptions of the same message. But for the above “textbook RSA”, if you encrypt the same message twice, you get back the same ciphertext (encryption is deterministic). This is typically fixed by introducing a padding scheme, but these are both complex, and easy to get wrong (there have been many attacks in practice on improper RSA padding).

So Textbook RSA is simple to describe, but moderately complex to implement in a way that is reasonably efficient for parameters people actually use, and quite complex to implement in a way that is known to be secure. RSA also has the issue that (even when properly padded) it is insecure against quantum computers, so once those develop further it’ll be insecure in all settings.

<h2>LWE</h2>

As we discussed before, Gaussian Elimination breaks noiseless LWE. How does introducing noise change things?

In [ ]:
import random
from utils import *

class LWEPrivKey:
    def __init__(self, n, q, B):
        self.n = n
        self.q = q
        self.B = B
    def key_gen(self):
        self.s = sample_unif_vector(self.n, self.q)
    def enc(self, m):
        # Sampling A, e
        A = sample_unif_matrix(n, self.q)
        e = sample_bounded_vector(self.n, self.B)
        # Computing b:= As + e
        b = matrix_vector_multiply(A, self.s, self.q)
        b = vector_vector_add(b, e, self.q)
        # Scaling m -> (q//2)m
        scaled_m = [(self.q//2)*m[i] % self.q for i in range(self.n)]
        # Adding (q//2)m to b = As + e
        b = vector_vector_add(b, scaled_m, self.q)
        return (A,b)
    def dec(self, ctxt):
        (A,b) = ctxt[0], ctxt[1]
        # Recomputing As
        As = matrix_vector_multiply(A, self.s, self.q)
        # Recovering scaled_m = b - As
        for i in range(self.n):
            b[i] = (b[i] - As[i]) % q
        # Scaling (q//2)m + e -> m
        m = [0 for _ in range(self.n)]
        for i in range(self.n):
            m[i] = round(b[i] / (self.q//2)) % 2
        return m

In [ ]:
class LWEPrivKey:
    def __init__(self, n, q, B):
        self.n = n
        self.q = q
        self.B = B
        
    def key_gen(self):
        A = sample_unif_matrix(self.n, self.q)
        self.sk = sample_bounded_vector(self.n, self.b)
        As = matrix_vector_multiply(A, self.sk, self.q)
        e = sample_bounded_vector(self.n, self.B)
        b = vector_vector_add(As, e, self.q)
        self.pk = (A,b)
        
    def enc(self, m):
        (A,b) = self.pk
        # Sampling r
        r = sample_bounded_vector(self.n, self.B)
        e_prime = sample_bounded_vector(self.n, self.B)
        # Computing u := A^tr + e'
        At = matrix_transpose(A)
        u = matrix_vector_multiply(At, r, self.q)
        u = vector_vector_add(u, e_prime, self.q)
        # Approximately computing r^t(As)
        rAs = vector_vector_inner_product(r, b, self.q)
        # Adding e_double_prime so it is a LWE encryption
        e_prime_prime = sample_bounded_vector(1, self.B)[0]
        v = rAs + e_prime_prime % self.q
        # Adding an encoding of m to the random pad
        v = (v + (self.q//2)*m) % self.q
        return (u,v)
    def dec(self, ctxt):
        (u, v) = ctxt[0], ctxt[1]
        s = self.sk
        # Approximately (r^tA)s
        rAs = vector_vector_inner_product(r, b, self.q)
        # Recomputing As
        As = matrix_vector_multiply(u, s, self.q)
        # Substracting rAs from v
        v = (v - rAs) % self.q
        # Deconding (q//2)*m + error_terms
        return round(v/(self.q//2)) % 2